In [1]:
import numpy as np
import scipy
from scipy import special
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
%matplotlib notebook
plt.style.use('seaborn-pastel')

In [9]:
lambd = 1
f0 = 1
L_x = 1
L_y = 1
m = 100
n = 100
h_x = L_x / m
h_y = L_y / n
x_i = np.arange(0, L_x+h_x, h_x)
y_j = np.arange(0, L_y+h_y, h_y)
r_i = len(x_i)
r_j = len(y_j)
g_y_x = np.zeros([r_j, r_i]) #сетка размером r_j * r_i
Temp = 300
for i in range(r_i): #гран условие при y = 0
    g_y_x[0][i] = 0
for j in range(r_j): #гран условие при x = X
    g_y_x[j][r_i-1] = Temp
print(g_y_x)
alph1 = np.zeros([r_i-1])
beta1 = np.zeros([r_i-1])
for s in range(100):
    # продольная прогонка
    for j in range(1,r_j-1):
        beta_N = lambd * h_x / h_y
        beta_W = lambd * h_y / h_x
        beta_S = lambd * h_x / h_y
        beta_E = lambd * h_y / h_x
        beta_P = beta_E + beta_N + beta_S + beta_W
        #нет потока и нулевая температура
        alph1[0] = 0
        beta1[0] = g_y_x[j][0]
        for i in range(1,r_i-1):
            alph1[i] = -beta_E / (beta_W * alph1[i - 1] - beta_P)
            beta1[i] = (-f0 * h_x * h_y -beta_N*g_y_x[j+1][i] - beta_S * g_y_x[j-1][i] - beta_W * beta1[i-1]) / (beta_W * alph1[i - 1] - beta_P)
        for m in range(r_i - 2, -1, -1):
            g_y_x[j+1][m] = g1_t_h[j+1][m + 1] * alph1[m] + beta1[m]
    # поперечная прогонка        
    for i in range(r_i):
        
            

[[  0.   0.   0. ...   0.   0. 300.]
 [  0.   0.   0. ...   0.   0. 300.]
 [  0.   0.   0. ...   0.   0. 300.]
 ...
 [  0.   0.   0. ...   0.   0. 300.]
 [  0.   0.   0. ...   0.   0. 300.]
 [  0.   0.   0. ...   0.   0. 300.]]
